# Passenger and Plane Itinerary Optimization (PPIO) Problem
----

## Problem Description: 

For a given list of passengers with predefined origin airports and final destination airports, a fixed number of planes in operation, and a fixed network topology $G(A,R)$, we want to extract the optimal schedule or itinerary for each plane $p$ and each passenger $h$ throughout the day. 

This problem is related to the class of Traveling Salesman Problems, where the goal is to find the shortest distance path through cities such that the salesman starts and stops at the same city. The key differences in this optimization problem is that we are interested in minimizing the total travel time over each passenger $h$, and we have multiple agents traversing the network at scheduled times.

We will solve this optimization problem using a multi-integer linear programming approach. There are many applications of this optimization problem including logistics planning, energy management, crew scheduling, and telecommunications network design.

## PPIO Model Formulation:

### Sets and Indices
1. Airports, denoted by $A$. Capacity of airport, $K_a$
2. Passengers, denoted by $H$.
3. Aircrafts, denoted by $P$. Capacity of aircraft $p$ is denoted by $C_p$.
4. Discretized times, denoted by $T$. Total number of time periods will be denoted by $|T|$.
5. Flights, denoted by $F$. These are pairs $(e,t)$ of edges and starting times. A flight, $f$, is a possible plane going along $e$, with starting time $t$ (at the origin airport of $e$).

### Decision Variables
1. For every $h \in H$, $f \in F$, let $X_{h,f}=1$  if and only if human $h$ takes the flight $f$.
2. For every $p \in P$, $f \in F$,  let $Y_{p,f}=1$ if and only if aircraft $p$ flies flight $f$.
3. For every $t \in T$, $a \in A$, $h \in H$, let $Z_{t,h,a} = 1$ if and only if human $h$ is at airport $a$ at time $t$.
4. For every $t \in T$, $a \in A$, $p \in P$, let $W_{t,p,a} = 1$ if and only if aircraft $p$ is at airport $a$ (waiting) at time $t$.$

### Number of Decision Variables
- Define the complete graph of an airline's network as $G(A,E)$, where A ~ set of airports,|A| = n , E ~ all possible routes (i,j)
- Suppose $|H| = k, |F| = n(n-1)(T)$, then we have $n*(n-1)*T*k$ total $X_{h,f}$ variables
- Suppose $|P| = m$, then we have $n*(n-1)*T*m$ total $Y_{p,f}$ variables
- We have $T*k*n$ total $Z_{t,h,a}$ variables
- We have $T*n*m$ total $W_{t,p,a}$ variables
- Total: $n(n-1)(k)T + n(n-1)(m)T + (k)(n)T + (n)(m)T$


### Objective Function
**Minimum Total Travel Time**. Minimize the total travel time across all passengers. Each passenger $h$ is assigned an itinerary consisting of sequential $Z_{t,h,a}$ and $X_{h,f}$ variables.
\begin{equation}
\text{Min} \sum_{h} D_{h}
\tag{1}
\end{equation}

where the duration for a passenger $h \in H$ is the number of discretized time intervals between the first instance of departing from the origin airpor and the first instance of arriving at the ,

\begin{equation}
D_{h} = |T| - \sum_{t} Z_{t,h,orig(h)} - \sum_{t} Z_{t,h,dest(h)}
\tag{2}
\end{equation}

<font color="red">Note: All passengers start at their respective origin airport at time 0 and must reach their final destination airport by time T-1</font>

Define functions on $h$  (These will be fixed and defined when initializing variables)
1. $orig(h)$ = origin airport for passenger $h$
2. $dest(h)$ = final destination airport for passenger $h$

**Alternative Objective Function**

\begin{equation}
\text{Max} \sum_{h} \sum_{t} [Z_{t,h,orig(h)} + Z_{t,h,dest(h)}]
\tag{3}
\end{equation}


#### Constraints
1) **Starting Boundary Conditions**
   
   At $t=0$, all passengers $h \in H$ begin at the origin airport:

   \begin{equation}
   \forall h \in H, Z_{0,h,orig(h)} = 1
   \tag{4}
   \end{equation}


   At $t=0$, all planes $p \in P$ begin at some starting airport (let this be a random assignment):

   \begin{equation}
   \forall p \in P, W_{0,p,orig(p)} = 1
   \tag{5}
   \end{equation}


2) **Ending Boundary Conditions**

   At $t=|T|-1$, all passengers $h \in H$ end at their final destination airport:

   \begin{equation}
   \forall h \in H, Z_{|T|-1,h,dest(h)} = 1
   \tag{6}
   \end{equation}


   At $t=|T|-1$, all planes $p \in P$ must end at their starting airport:

   \begin{equation}
   \forall p \in P, W_{|T-1|,p,orig(p)} = 1
   \tag{7}
   \end{equation}

3) **Plane Capacity**

   Every plane $p$ has a passenger capacity: total capacity of all assigned planes or 0 if no planes is assigned to $f$:

   \begin{equation}
   \forall f \in F, \sum_{h} X_{h,f} \leq \sum_{p} C_{p} Y_{p,f}
   \tag{8}
   \end{equation}


4) **Airport Capacity**

   Every airport $a$ has a plane capacity: total capacity of all defined airports or 0 if no planes is assigned to $f$:

   \begin{equation}
   \forall a \in A, t \in T, \sum_{p} W_{t,p,a} \leq K_{a}
   \tag{9}
   \end{equation}
   

5) **Conflicting Flights for planes and passengers**

   A particulary plane $p$ cannot be assigned to more than one flight (at the exact same time $t$). For every pair of flights $f_{1}, f_{2}$, both of these flights cannot be serviced by the same plane $p$ because the time intervals for each $f$ intersect. 

   \begin{equation}
   \forall (f_{i}, f_{j}) \, \, \, Y_{p,f_{1}} + Y_{p,f_{2}} \leq 1
   \tag{8}
   \end{equation}

   For every passenger and pair of conflicting flights, $f_{1}, f_{2}$:

   \begin{equation}
   \forall (f_{i}, f_{j}), h \in H, \, \, \, X_{h,f_{1}} + X_{h,f_{2}} \leq 1
   \tag{9}
   \end{equation}

   The conficting flights, with respective to time $t$ is defined as:

   start($f_{1}$) $\leq$ start($f_{2}$) $\leq$ end($f_{1}$)    OR    start($f_{2}$) $\leq$ start($f_{1}$) $\leq$ end($f_{2}$)

6) **Conservation Laws for planes and passengers**

   A passenger at any given time $t$ must be either on a flight or waiting at an airport but they cannot be waiting at an airport or on a flight at the same time:

   \begin{equation}
   \forall h \in H, t \in T, \, \, \, \sum_{a} Z_{t,h,a} + \sum_{f: start(f) \leq t \leq end(f)} X_{h,f} = 1
   \tag{10}
   \end{equation}


   Similarly, a plane at any given time $t$ must be either operating a flight or waiting at an airport but they cannot be waiting at an airport or on a flight at the same time:

   \begin{equation}
   \forall p \in P, t \in T, \, \, \, \sum_{a} W_{t,p,a} + \sum_{f: start(f) \leq t \leq end(f)} Y_{p,f} = 1
   \tag{11}
   \end{equation}






7) **Continuity of Origin Airport**
   
   For every plane $p$ and flight $f$, the plane must be at the origin airport of the flight before the start time of $f$:

   \begin{equation}
   \forall p \in P, f \in F \, \, \, \,  W_{start(f) - 1, p, orig(f)} \geq Y_{p,f}
   \tag{12}
   \end{equation}

   This constraint ensures that if a plane is operating route $f$ then that means it must have been waiting at the origin airport of $f$ at the time right before  (both binary variables are stored as 1). 

   This same continuity principle exists for passengers $h \in H$. If a passenger is on some flight $f$, then they must have been waiting at the corresponding origin airport at a time $t-1$:

   \begin{equation}
   \forall h \in H, f \in F \, \, \, Z_{start(f) - 1, h, orig(f)} \geq X_{h,f}
   \tag{13}
   \end{equation}

8) **Continuity of Destination Airport**

    Similar to the previous constraint but now in the opposite direction. For every plane $p$ and flight $f$, the plane must be at the destination airport of the flight after the end time of the flight. For every plane $p$, time $t$, and airport $a$, if $W_{t,p,a} = 1$, then it must have been at that airport or a flight was flown there (just arrived):

    \begin{equation}
    \forall p \in P, t \in T , a \in A \, \, \, \,  W_{t - 1, p, a} + \sum_{f: dest(f)=a, end(f)=t-1} Y_{p,f} \geq W_{t,p,a}
    \tag{14}
    \end{equation}

    Similarly, the continuity principle exists for all passengers through the network:

    \begin{equation}
    \forall h \in H, t \in T , a \in A \, \, \, \,  Z_{t - 1, h, a} + \sum_{f: dest(f)=a, end(f)=t-1} X_{h,f} \geq Z_{t,h,a}
    \tag{15}
    \end{equation}

9. <font color="red">  **Assigning Passengers to only Active Flights**
    
    kp: possible constraint we need to tie passengers with planes? 

    Assignment of passengers to flights: A passenger can only be assigned to a flight if and only if a plane is operating that flight: 

    \begin{equation}
    \forall h \in H, f \in F \, \, \, \,  X_{h,f} \leq Y_{p,f}
    \tag{16}
    \end{equation}


</font>


In [113]:
# import packages
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import truncnorm 
import folium
import json
import os
from folium import plugins
import gurobipy as gp
import random
import itertools
import math
from gurobipy import GRB
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Data Preprocessing

Large-scale Air Traffic and Passenger Data
1. Considering only domestic US air travel
2. ~5,000 total US airports
3. ~45,000 daily domestic flights in the US
4. 1.73 million passengers transported daily

Sample Passenger Demand Data
1. We have passenger data extracted from BTS (Bureau of Transportation Statistics)
2. This passenger table contains a the variable h indexed by integers: $[h_{1}, h_{2}, . . . ]$
3. The origin and destination functions are applied to each passenger $h_{i}$ and are assigned to the two respective columns

In [114]:
# define set of airports in network
set_of_airports = ['EWR', 'BWI', 'LAX', 'ORD', 'SFO', 'SEA', 'LAS', 'MCO']
# Newark, Baltimore, Los Angeles, San Francisco, Seattle, Las Vegas, Orlando
# number of airports N
N = len(set_of_airports)
# define the set of planes
planes_p = [f'''p{x}''' for x in range(P)]

def plane_capacity(planes, mean, std):
    # define capacity of each plane
    # src: https://www.faa.gov/sites/faa.gov/files/regulations_policies/policy_guidance/benefit_cost/econ-value-section-3-capacity.pdf  
    capacity_p = {}
    # define distribution for plane capacity 
    # truncated normal distribution  (support: {85, 215}) with mean 150 and stdev 80
    a, b = (85 - mean) / std, (215 - mean) / std
    for i in planes:
        capacity_p[i] = int(truncnorm.rvs(a, b, loc = mean, scale = std, size=1))
    return capacity_p

def airport_capacity(airports, mean, std):
    # define capacity of each plane
    capacity_a = {}
    # define distribution for airport capacity 
    # truncated normal distribution  (support: {85, 215}) with mean 150 and stdev 80
    a, b = (85 - mean) / std, (215 - mean) / std
    for i in airports:
        capacity_a[i] = int(truncnorm.rvs(a, b, loc = mean, scale = std, size=1))
    return capacity_a

def create_graph(airports):
    # consider a complete graph (all possible options)
    G_complete = nx.complete_graph(set_of_airports) # we select flights (t + route(i,j)) to operate these   (a flight object is later defined)
    coords = {}
    for i in G_complete.nodes:
        origin_lat = float(airports_data.loc[airports_data['local_code'] == i]['latitude_deg'].values)
        origin_long = float(airports_data.loc[airports_data['local_code'] == i]['longitude_deg'].values)
        coords[i] = (origin_lat, origin_long)
    nx.set_node_attributes(G_complete, coords, name = 'pos')
    G_complete = G_complete.to_directed()
    return G_complete

# DATA: airports IATA Code and Geographical Coordinates
airports_data = pd.read_csv('Data/us-airports.csv')
airports_data = airports_data[['latitude_deg','longitude_deg','local_code']]
airports_data = airports_data.iloc[1:]

# DATA: on time preformance by airlines between two airports : extract distance and duration
us_ontime_market = pd.read_csv('Data/T_ONTIME_MARKETING.csv')
us_ontime_market = us_ontime_market.groupby(['ORIGIN', 'DEST'])[['ACTUAL_ELAPSED_TIME', 'DISTANCE']].mean().reset_index()

In [116]:
# Create a map centered around the US
m = folium.Map(location=[39.8283, -98.5795], zoom_start=5)
G = create_graph(set_of_airports)
# Iterate through the nodes and add markers for each airport
coords = nx.get_node_attributes(G, "pos")
for node in G.nodes:
    lat, lon = nx.get_node_attributes(G, 'pos')[node]
    folium.Marker(location=[lat, lon], popup=node).add_to(m)
# Iterate through the edges and draw lines connecting airports
for u, v in G.edges():
    lat1, lon1 = G.nodes[u]["pos"]
    lat2, lon2 = G.nodes[v]["pos"]
    folium.PolyLine([(lat1, lon1), (lat2, lon2)], color="blue").add_to(m)

In [132]:
# define an object called Flight
class Flight:
    def __init__(self, origin, destination, departure_time):
        self.origin = origin
        self.destination = destination
        self.departure_time = departure_time
        self.origin_lat = float(airports_data.loc[airports_data['local_code'] == origin]['latitude_deg'].values)
        self.origin_long = float(airports_data.loc[airports_data['local_code'] == origin]['longitude_deg'].values)
        self.dest_lat = float(airports_data.loc[airports_data['local_code'] == destination]['latitude_deg'].values)
        self.dest_long = float(airports_data.loc[airports_data['local_code'] == destination]['longitude_deg'].values)

    def get_origin(self):
        return self.origin

    def get_destination(self):
        return self.destination
        
    def get_departure_time(self):
        return self.departure_time


    def get_fight_duration(self):
        filtrd_data = us_ontime_market[(us_ontime_market['ORIGIN'] == self.origin) & (us_ontime_market['DEST'] == self.destination)]
        flight_duration = int(filtrd_data['ACTUAL_ELAPSED_TIME'].values[0])
        return flight_duration


    def get_arrival_time(self):
        return self.get_fight_duration() + self.departure_time


    def get_distance(self):
        # Radius of the Earth in miles
        earth_radius = 3958.8
        # Convert latitude and longitude from degrees to radians
        lat1, lon1, lat2, lon2 = map(math.radians, [self.origin_lat, self.origin_long, self.dest_lat, self.dest_long])
        # Haversine formula
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = earth_radius * c
        distance = us_ontime_market[(us_ontime_market['ORIGIN'] == self.origin) & (us_ontime_market['DEST'] == self.destination)]['DISTANCE']
        return distance 
        

    def visualize_route(self):
        m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
        folium.Marker(location=[self.origin_lat, self.origin_long], popup=node).add_to(m)
        folium.Marker(location=[self.dest_lat, self.dest_long], popup=node).add_to(m)
        folium.PolyLine([(self.origin_lat, self.origin_long), (self.dest_lat, self.dest_long)], color="blue").add_to(m)
        return m


    def __str__(self):
        return f"Flight from {self.origin} to {self.destination} departing at t = {self.departure_time}"

### Define all Decision Variables and load into Gurobi

In [137]:
# define Time -  Assume a 18 hour period descritized by time intervals by the minute 
T = np.arange(0, 18*60, 1)

set_of_airports = ['EWR', 'BWI', 'LAX', 'ORD', 'SFO', 'SEA', 'LAS', 'MCO']
# Newark, Baltimore, Los Angeles, San Francisco, Seattle, Las Vegas, Orlando

# number of airports N
N = len(set_of_airports)

# define the set of planes and P number of planes to use
P = 5
planes_p = [f'''p{x}''' for x in range(P)]

capacity_p = plane_capacity(planes_p, 50, 23)
capacity_a = airport_capacity(set_of_airports)

# define all possible flights: this is dependent on the time AND route 
for i in range(T):
    for j in 

In [143]:
import itertools

# Given list
my_list = [1, 2, 3]

# Get all permutations as tuples
permutations = list(itertools.permutations(my_list, 2))

print(permutations)

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]


In [147]:
# randomly select routes from complete graph
import random


# Randomly select tuples
selected_tuples = random.sample(permutations, 2)

print(selected_tuples)


[(1, 3), (3, 1)]


In [ ]:
# define all the variables 

# Create a Gurobi model
model = gp.Model("Flight_Optimization")

# Create a dictionary to hold the decision variables X[f, h] for each combination of flight and passenger
X = {}
for f in list_of_flights:  # Replace list_of_flights with your list of Flight objects
    for h in list_of_passengers:  # Replace list_of_passengers with your list of passengers
        X[f, h] = model.addVar(vtype=gp.GRB.BINARY, name=f"X_{f}_{h}")

# Update the model to include the variables
model.update()

# Now you can use X to represent the decision variables for flight assignments.


In [18]:
# pull the data
humans_data = pd.read_csv('Data/Airline Dataset Updated - v2.csv')